In [1]:
# !pip install diffusers==0.10.0 transformers ftfy opencv-python ipywidgets matplotlib accelerate "huggingface_hub<0.25.0" -q
# !pip install transformers==4.49.0
import os 
import numpy as np
import argparse
import json
from PIL import Image

import torch
import random

from models.p2p_editor import P2PEditor



def mask_decode(encoded_mask,image_shape=[512,512]):
    length=image_shape[0]*image_shape[1]
    mask_array=np.zeros((length,))
    
    for i in range(0,len(encoded_mask),2):
        splice_len=min(encoded_mask[i+1],length-encoded_mask[i])
        for j in range(splice_len):
            mask_array[encoded_mask[i]+j]=1
            
    mask_array=mask_array.reshape(image_shape[0], image_shape[1])
    # to avoid annotation errors in boundary
    mask_array[0,:]=1
    mask_array[-1,:]=1
    mask_array[:,0]=1
    mask_array[:,-1]=1
            
    return mask_array


def setup_seed(seed=1234):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


C:\Users\Hirundo\.conda\envs\p2p\lib\site-packages\accelerate\utils\torch_xla.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


In [2]:
import torch
print(torch.version.cuda)

11.7


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Using device: cuda
GPU: NVIDIA GeForce RTX 4070
GPU memory: 12.9 GB


In [4]:
import os, sys, subprocess
REPO = r"C:\Users\Hirundo\Documents\GitHub\PnPInversion"
DATA = r"H:\PIE-Bench\data"
OUT  = r"H:\PIE-Bench\output_tuned"

os.chdir(REPO)

cmd = [
    sys.executable, "-u", "run_editing_masactrl.py",
    "--data_path", DATA,
    "--output_path", OUT,
    "--edit_category_list", "0",   # start with one category to sanity-check
    "--edit_method_list", "ddim+masactrl",
    "--rerun_exist_images"
]
print("Running:\n", " ".join(cmd))
res = subprocess.run(cmd, capture_output=True, text=True)
print(res.stdout)
print(res.stderr)
if res.returncode != 0:
    raise RuntimeError(f"Script failed with code {res.returncode}")

Running:
 C:\Users\Hirundo\.conda\envs\p2p\python.exe -u run_editing_masactrl.py --data_path H:\PIE-Bench\data --output_path H:\PIE-Bench\output_tuned --edit_category_list 0 --edit_method_list ddim+masactrl --rerun_exist_images
editing image [H:\PIE-Bench\data/annotation_images\0_random_140/000000000000.jpg] with [ddim+masactrl]
input text embeddings : torch.Size([1, 77, 1024])
latents shape:  torch.Size([1, 4, 64, 64])
Valid timesteps:  tensor([  1,  34,  67, 100, 133, 166, 199, 232, 265, 298, 331, 364, 397, 430,
        463, 496, 529, 562, 595, 628, 661, 694, 727, 760, 793, 826, 859, 892,
        925, 958])
input text embeddings : torch.Size([1, 77, 1024])
latents shape:  torch.Size([1, 4, 64, 64])
MasaCtrl at denoising steps:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
MasaCtrl at U-Net layers:  [10, 11, 12, 13, 14, 15]
input text embeddings : torch.Size([2, 77, 1024])
latents shape:  tor

In [5]:
import os, sys, subprocess

# Point to your MASACTRL env Pytho


REPO = r"C:\Users\Hirundo\Documents\GitHub\PnPInversion"
DATA = r"H:\PIE-Bench\data"
OUT  = r"H:\PIE-Bench\output_tuned"

os.chdir(REPO)

cmd = [
    sys.executable, "-u", "run_editing_masactrl.py",
    "--data_path", DATA,
    "--output_path", OUT,
    "--edit_category_list", "1", "2","3", "4","5","6", "7","8","9",  # start with one category to sanity-check
    "--edit_method_list", "ddim+masactrl",
    "--rerun_exist_images"
]
print("Running:\n", " ".join(cmd))
res = subprocess.run(cmd, capture_output=True, text=True)
print(res.stdout)
print(res.stderr)
if res.returncode != 0:
    raise RuntimeError(f"Script failed with code {res.returncode}")

Running:
 C:\Users\Hirundo\.conda\envs\p2p\python.exe -u run_editing_masactrl.py --data_path H:\PIE-Bench\data --output_path H:\PIE-Bench\output_tuned --edit_category_list 1 2 3 4 5 6 7 8 9 --edit_method_list ddim+masactrl --rerun_exist_images


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [6]:
from pathlib import Path
from PIL import Image
import os
ROOT = r"H:\PIE-Bench\output_tuned\ddim+masactrl\annotation_images"
TARGET = (512, 512)
EXTS = (".jpg", ".jpeg", ".png")   # add more if needed

n_total = n_cropped = n_resized = 0

for p in Path(ROOT).rglob("*"):
    if not p.is_file() or p.suffix.lower() not in EXTS:
        continue
    n_total += 1

    im = Image.open(p).convert("RGB")
    w, h = im.size

    # Heuristic: montage strip (4× panels horizontally, 512 tall)
    if h == 512 and w >= 2000:
        im = im.crop((w - 512, 0, w, 512))  # rightmost panel = edited
        n_cropped += 1
    # Otherwise, just enforce target size
    elif (w, h) != TARGET:
        im = im.resize(TARGET, Image.LANCZOS)
        n_resized += 1

    im.save(p)

print(f"Processed: {n_total} files  |  cropped: {n_cropped}  |  resized: {n_resized}")


Processed: 700 files  |  cropped: 700  |  resized: 0


In [ ]:
import os, sys, runpy, glob

# 1) point to your repo root and outputs
REPO = r"C:\Users\Hirundo\Documents\GitHub\PnPInversion"   # repo root (has evaluation/)
OUT = r"C:\Users\Hirundo\Documents\GitHub\PnPInversion"  # <--- CHANGED: Set to the root folder of the benchmark data
DATA = r"H:\PIE-Bench\data"                   # folder that contains mapping_file.json
CATS = ["0","1","2","3","4","5","6","7","8","9"]                                # or all: ["0","1","2","3","4","5","6","7","8","9"]
METHOD = "1_ddim+masactrl"  # <--- CHANGED

os.chdir(REPO)

cmd = [
    sys.executable, "-u", "-m", "evaluation.evaluate",
    "--annotation_mapping_file", os.path.join(DATA, "mapping_file.json"),
    "--src_image_folder", os.path.join(OUT, "annotation_images"),
   "--tgt_methods", METHOD,
    "--edit_category_list", *CATS,
    "--metrics",
    "structure_distance", "psnr_unedit_part", "lpips_unedit_part", "mse_unedit_part", "ssim_unedit_part",
    "clip_similarity_source_image",
    "clip_similarity_target_image",
    "clip_similarity_target_image_edit_part",
    "--result_path", os.path.join(OUT, "eval_masactrl.csv"),
    # optional:
    "--device", "cuda"
]
res = subprocess.run(cmd, capture_output=True, text=True)
print(res.stdout)
print(res.stderr)
res.check_returncode()

In [8]:
import os, sys, runpy, glob

# 1) point to your repo root and outputs
REPO = r"C:\Users\Hirundo\Documents\GitHub\PnPInversion"   # repo root (has evaluation/)
OUT = r"H:\PIE-Bench"  # <--- CHANGED: Set to the root folder of the benchmark data
DATA = r"H:\PIE-Bench\data"                   # folder that contains mapping_file.json
CATS = ["0","1","2","3","4","5","6","7","8","9"]                                # or all: ["0","1","2","3","4","5","6","7","8","9"]
METHOD = "1_ddim+masactrl"  # <--- CHANGED

os.chdir(REPO)

cmd = [
    sys.executable, "-u", "-m", "evaluation.evaluate",
    "--annotation_mapping_file", os.path.join(DATA, "mapping_file.json"),
    "--src_image_folder", os.path.join(OUT, "annotation_images"),
   "--tgt_methods", METHOD,
    "--edit_category_list", *CATS,
    "--metrics",
    "structure_distance", "psnr_unedit_part", "lpips_unedit_part", "mse_unedit_part", "ssim_unedit_part",
    "clip_similarity_source_image",
    "clip_similarity_target_image",
    "clip_similarity_target_image_edit_part",
    "--result_path", os.path.join(OUT, "eval_masactrlsd2_new.csv"),
    # optional:
     "--device", "cuda"
]
res = subprocess.run(cmd, capture_output=True, text=True)
print(res.stdout)
print(res.stderr)
res.check_returncode()

evaluating image 000000000000 ...
evluating method: 1_ddim+masactrl
evluating metric: structure_distance
evluating metric: psnr_unedit_part
evluating metric: lpips_unedit_part
evluating metric: mse_unedit_part
evluating metric: ssim_unedit_part
evluating metric: clip_similarity_source_image
evluating metric: clip_similarity_target_image
evluating metric: clip_similarity_target_image_edit_part
evaluating image 000000000001 ...
evluating method: 1_ddim+masactrl
evluating metric: structure_distance
evluating metric: psnr_unedit_part
evluating metric: lpips_unedit_part
evluating metric: mse_unedit_part
evluating metric: ssim_unedit_part
evluating metric: clip_similarity_source_image
evluating metric: clip_similarity_target_image
evluating metric: clip_similarity_target_image_edit_part
evaluating image 000000000002 ...
evluating method: 1_ddim+masactrl
evluating metric: structure_distance
evluating metric: psnr_unedit_part
evluating metric: lpips_unedit_part
evluating metric: mse_unedit_par

CalledProcessError: Command '['C:\\Users\\Hirundo\\.conda\\envs\\p2p\\python.exe', '-u', '-m', 'evaluation.evaluate', '--annotation_mapping_file', 'H:\\PIE-Bench\\data\\mapping_file.json', '--src_image_folder', 'H:\\PIE-Bench\\annotation_images', '--tgt_methods', '1_ddim+masactrl', '--edit_category_list', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '--metrics', 'structure_distance', 'psnr_unedit_part', 'lpips_unedit_part', 'mse_unedit_part', 'ssim_unedit_part', 'clip_similarity_source_image', 'clip_similarity_target_image', 'clip_similarity_target_image_edit_part', '--result_path', 'H:\\PIE-Bench\\eval_masactrlsd2_new.csv', '--device', 'cuda']' returned non-zero exit status 1.

In [ ]:
import numpy as np
from PIL import Image

src = r"H:\PIE-Bench\data\annotation_images\0_random_140\000000000000.jpg"
edt = r"H:\PIE-Bench\output\ddim+masactrl\annotation_images\0_random_140\000000000000.jpg"

S = np.array(Image.open(src).convert("RGB"))
E = np.array(Image.open(edt).convert("RGB"))

# load/compute your 512x512 mask (1=edited, 0=unedited)
# mask = ... (np.uint8, 0/1 shape (512,512))

diff_unedited = (S - E)[mask == 0]
print("MSE on unedited region:", np.mean(diff_unedited.astype(np.float32)**2))
print("All equal on unedited region:", np.all(diff_unedited == 0))

In [ ]:
import sys, subprocess

pkgs = [
    "huggingface_hub==0.14.1",   # accepts use_auth_token
    "transformers==4.30.2",      # compatible with hub 0.14.x
    "torchmetrics==0.11.4",      # you asked for this; works with Lightning 2.0.x
    "lpips",                     # needed for LPIPS metric
    "timm",                      # needed if you use structure_distance (DINO ViT)
    "open-clip-torch"           # (optional) not used by torchmetrics.CLIPScore, but handy
]
subprocess.run([sys.executable, "-m", "pip", "install", *pkgs], check=True)

In [ ]:
from torchmetrics.multimodal import CLIPScore
metric = CLIPScore(model_name_or_path="openai/clip-vit-large-patch14")
print("CLIPScore ready.")


In [ ]:
p2p_editor=P2PEditor(["null-text-inversion+p2p","directinversion+p2p"], torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') )

In [ ]:
torch.cuda.empty_cache()
allocated_memory = torch.cuda.memory_allocated() / (1024**3)
print(f"PyTorch Allocated Memory: {allocated_memory:.2f} GB")

In [ ]:
# null-text-inversion+p2p
torch.cuda.empty_cache()



edited_image = p2p_editor("null-text-inversion+p2p",
                    image_path="scripts/example_cake.jpg",
                    prompt_src="a round cake with orange frosting on a wooden plate",
                    prompt_tar="a square cake with orange frosting on a wooden plate",
                    guidance_scale=7.5,
                    cross_replace_steps=0.4,
                    self_replace_steps=0.6,
                    blend_word=((("cake", ),
                                ("cake", ))),
                    eq_params={
                        "words": ("cake", ),
                        "values": (2, )
                    },
                    proximal="l0",
                    quantile=0.75,
                    use_inversion_guidance=True,
                    recon_lr=1,
                    recon_t=200,
                    )

print("null-text-inversion+p2p")
display(edited_image)

In [ ]:
# directinversion(ours)+p2p

edited_image = p2p_editor("directinversion+p2p",
                    image_path="scripts/example_cake.jpg",
                    prompt_src="a round cake with orange frosting on a wooden plate",
                    prompt_tar="a square cake with orange frosting on a wooden plate",
                    guidance_scale=7.5,
                    cross_replace_steps=0.4,
                    self_replace_steps=0.6,
                    blend_word=((("cake", ),
                                ("cake", ))),
                    eq_params={
                        "words": ("cake", ),
                        "values": (2, )
                    },
                    proximal="l0",
                    quantile=0.75,
                    use_inversion_guidance=True,
                    recon_lr=1,
                    recon_t=100,
                    )

print("directinversion(ours)+p2p")
display(edited_image)

In [ ]:
# null-text-inversion+p2p

edited_image = p2p_editor("null-text-inversion+p2p",
                    image_path="scripts/example_cat.jpg",
                    prompt_src="a cat sitting on a table with a green eyes",
                    prompt_tar="a watercolor of a cat sitting on a table with a green eyes",
                    guidance_scale=7.5,
                    cross_replace_steps=0.4,
                    self_replace_steps=0.6,
                    blend_word=((("cat", ),
                                ("cat", ))),
                    eq_params={
                        "words": ("watercolor", ),
                        "values": (5, )
                    },
                    proximal="l0",
                    quantile=0.75,
                    use_inversion_guidance=True,
                    recon_lr=1,
                    recon_t=400,
                    )

print("null-text-inversion+p2p")
display(edited_image)

In [ ]:
# directinversion+p2p

edited_image = p2p_editor("directinversion+p2p",
                    image_path="scripts/example_cat.jpg",
                    prompt_src="a cat sitting on a table with a green eyes",
                    prompt_tar="a watercolor of a cat sitting on a table with a green eyes",
                    guidance_scale=7.5,
                    cross_replace_steps=0.4,
                    self_replace_steps=0.6,
                    blend_word=((("cat", ),
                                ("cat", ))),
                    eq_params={
                        "words": ("watercolor", ),
                        "values": (5, )
                    },
                    proximal="l0",
                    quantile=0.75,
                    use_inversion_guidance=True,
                    recon_lr=1,
                    recon_t=400,
                    )

print("directinversion(ours)+p2p")
display(edited_image)